# Playing Differentiated Demand Environemnt

## Imports and initialize


In [1]:
#Imports

from marketsai.markets.diff_demand import DiffDemand

#import ray

from ray import tune, shutdown, init
from ray.tune.registry import register_env
from ray.tune.integration.mlflow import MLflowLoggerCallback 
from ray.rllib.utils.schedules.exponential_schedule import ExponentialSchedule

import random
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import logging

/Users/matiascovarrubias/Library/Caches/pypoetry/virtualenvs/marketsai-reVLCGV_-py3.8/lib/python3.8/site-packages/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
Instructions for updating:
non-resource variables are not supported in the long term
{'agent_0': array([1.6, 1.6], dtype=float32), 'agent_1': array([1.6, 1.6], dtype=float32)} {'agent_0': 0.27249238, 'agent_1': 0.27249238} {'__all__': False} {'agent_0': 1.6, 'agent_1': 1.6}


In [2]:
# STEP 0: Inititialize ray
NUM_CPUS = 9
shutdown()
init(num_cpus=NUM_CPUS, 
    logging_level=logging.ERROR,
)

{'node_ip_address': '192.168.1.202',
 'raylet_ip_address': '192.168.1.202',
 'redis_address': '192.168.1.202:6379',
 'object_store_address': '/tmp/ray/session_2021-04-18_16-43-10_620065_94093/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2021-04-18_16-43-10_620065_94093/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2021-04-18_16-43-10_620065_94093',
 'metrics_export_port': 64421,
 'node_id': '37aa7d5ab63e88905f1b2f69ac0deef10fd082cd714bdb448dbfbfea'}

In [3]:
# STEP 1: register environment
register_env("diffdemand", DiffDemand)
env = DiffDemand()
policy_ids = [f"policy_{i}" for i in range(env.n_agents)]

## Experminet Configuration

In [4]:
# STEP 2: Experiment configuration

#Experiment configuration
test = True
date = "April14"
env_label = "DiffDd"
if (test==True): 
    MAX_STEPS = 40 * 1000
    exp_label =env_label+"_test_"+date 
else: 
    MAX_STEPS = 3000 * 1000
    exp_label =env_label+"_run_"+date 

verbosity = 2
stop = {"episodes_total": MAX_STEPS//100}

#Environment configuration
PRICE_BAND_WIDE = 0.1
LOWER_PRICE = 1.47 - PRICE_BAND_WIDE
HIGHER_PRICE = 1.93 + PRICE_BAND_WIDE
DEC_RATE = float(math.e ** (-4 * 10 ** (-6)))
DEC_RATE_HIGH = float(math.e ** (-4 * 10 ** (-6) * 4))

env_config = {
    "mkt_config": {
    #"lower_price": [LOWER_PRICE for i in range(env.n_agents)],
    #"higher_price": [HIGHER_PRICE for i in range(env.n_agents)],
    "parameteres": {
                "cost": [1 for i in range(env.n_agents)],
                "values": [2 for i in range(env.n_agents)],
                "ext_demand": 0,
                "substitution": 0.25,
    },
    "space_type": "MultiDiscrete",
    "gridpoints": 16,
    }
}

exploration_config = {"type": "EpsilonGreedy",
    "epsilon_schedule": ExponentialSchedule(
      schedule_timesteps = 1,
      framework="Torch",
      initial_p=1.0,
      decay_rate=DEC_RATE,
    ),
}

training_config = {
    "gamma": 0.95,
    "lr": 0.15,
    "env": "diffdemand",
    "exploration_config": exploration_config,
    "env_config": env_config,
    "horizon": 100,
    "soft_horizon": True,
    "no_done_at_end": True,
    "multiagent": {
        "policies": {
            policy_ids[i]: (
                None,
                env.observation_space["agent_{}".format(i)],
                env.action_space["agent_{}".format(i)],
                {},
            )
            for i in range(env.n_agents)
        },
        "policy_mapping_fn": (lambda agent_id: policy_ids[int(agent_id.split("_")[1])]),
    },
    "framework": "torch",
    "num_workers": NUM_CPUS - 1,
    "num_gpus": 0,
    "timesteps_per_iteration": 1000,
    "normalize_actions": False,
}


#stop = {"training_iteration": MAX_STEPS//1000}
#stop = {"info/num_steps_trained": MAX_STEPS}

# # Experiments

In [5]:
#DQN Methods: DQN, APEX, R2D2

# algo_list = ["DQN", "APEX", "R2D2"] 
algo_list=["DQN"]
for i in range(len(algo_list)):
    exp_name = exp_label+algo_list[i]
    results = tune.run(
        algo_list[i],
        name=exp_name,
        config=training_config,
        #checkpoint_freq=250,
        checkpoint_at_end=True,
        stop=stop,
        callbacks=[MLflowLoggerCallback(experiment_name=exp_name, save_artifact=True)],
        verbose=verbosity
    )


training_config_RBW=training_config.copy()
training_config_RBW["n_step"] = 5
training_config_RBW["noisy"] = True
training_config_RBW["num_atoms"] = 10
training_config_RBW["v_min"] = 0.5
training_config_RBW["v_min"] = 2


exp_name = exp_label+"RAINBOW"
results = tune.run(
    "DQN",
    name=exp_name,
    config=training_config_RBW,
    #checkpoint_freq=250,
    checkpoint_at_end=True,
    stop=stop,
    callbacks=[MLflowLoggerCallback(experiment_name=exp_name, save_artifact=True)],
    verbose=verbosity
)


(pid=94221) /Users/matiascovarrubias/Library/Caches/pypoetry/virtualenvs/marketsai-reVLCGV_-py3.8/lib/python3.8/site-packages/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
(pid=94221)   warnings.warn(msg)
(pid=94221) WARNING:tensorflow:From /Users/matiascovarrubias/Library/Caches/pypoetry/virtualenvs/marketsai-reVLCGV_-py3.8/lib/python3.8/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=94221) Instructions for updating:
(pid=94221) non-resource variables are not supported in the long term
(pid=94221) 2021-04-18 16:43:15,832	INFO trainer.py:641 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=94221) {'agent_0': array([1.6, 1.6], dtype=float32), 'agen

Trial DQN_diffdemand_b1bce_00000 reported episode_reward_max=37.374181758612394,episode_reward_min=29.370103059336543,episode_reward_mean=33.39382791464838,episode_len_mean=100.0,episodes_this_iter=8,policy_reward_min={'policy_0': 14.137089237570763, 'policy_1': 14.078079843893647},policy_reward_max={'policy_0': 18.850383492186666, 'policy_1': 19.76125978305936},policy_reward_mean={'policy_0': 16.688036227443565, 'policy_1': 16.705791687204812},custom_metrics={},sampler_perf={'mean_env_wait_ms': 0.14533029528590927, 'mean_raw_obs_processing_ms': 0.2386217519007965, 'mean_inference_ms': 2.067082925373489, 'mean_action_processing_ms': 0.08068274984657879},off_policy_estimator={},num_healthy_workers=8,timers={'learn_time_ms': 10.925, 'learn_throughput': 2929.086, 'update_time_ms': 2.157},info={'learner': {'policy_0': {'allreduce_latency': 0.0, 'grad_gnorm': array(0.00736521, dtype=float32), 'cur_lr': 0.15, 'mean_q': 3.852165699005127, 'min_q': 3.669006824493408, 'max_q': 3.939450025558471

Trial DQN_diffdemand_b1bce_00000 reported episode_reward_max=37.404665326699615,episode_reward_min=29.208372473716736,episode_reward_mean=33.50142038080164,episode_len_mean=100.0,episodes_this_iter=8,policy_reward_min={'policy_0': 14.137089237570763, 'policy_1': 14.078079843893647},policy_reward_max={'policy_0': 19.064815275371075, 'policy_1': 19.76125978305936},policy_reward_mean={'policy_0': 16.686001497088, 'policy_1': 16.81541888371364},custom_metrics={},sampler_perf={'mean_env_wait_ms': 0.14520244400031376, 'mean_raw_obs_processing_ms': 0.23926913414348142, 'mean_inference_ms': 2.061349107969603, 'mean_action_processing_ms': 0.08039292313885005},off_policy_estimator={},num_healthy_workers=8,timers={'learn_time_ms': 10.708, 'learn_throughput': 2988.347, 'update_time_ms': 2.235},info={'learner': {'policy_0': {'allreduce_latency': 0.0, 'grad_gnorm': array(0.00451011, dtype=float32), 'cur_lr': 0.15, 'mean_q': 4.200776100158691, 'min_q': 4.070622444152832, 'max_q': 4.318135738372803, '

Trial DQN_diffdemand_b1bce_00000 reported episode_reward_max=38.27290922217071,episode_reward_min=29.208372473716736,episode_reward_mean=33.792624291144314,episode_len_mean=100.0,episodes_this_iter=8,policy_reward_min={'policy_0': 13.927396217361093, 'policy_1': 14.438492741435766},policy_reward_max={'policy_0': 19.682217812165618, 'policy_1': 19.721824208274484},policy_reward_mean={'policy_0': 16.84685320606455, 'policy_1': 16.94577108507976},custom_metrics={},sampler_perf={'mean_env_wait_ms': 0.14454672749519765, 'mean_raw_obs_processing_ms': 0.239383418317075, 'mean_inference_ms': 2.049879462617832, 'mean_action_processing_ms': 0.07970275583135597},off_policy_estimator={},num_healthy_workers=8,timers={'learn_time_ms': 11.303, 'learn_throughput': 2831.162, 'update_time_ms': 2.327},info={'learner': {'policy_0': {'allreduce_latency': 0.0, 'grad_gnorm': array(0.02817149, dtype=float32), 'cur_lr': 0.15, 'mean_q': 4.268095016479492, 'min_q': 4.0373029708862305, 'max_q': 4.374894618988037,

Trial DQN_diffdemand_b1bce_00000 reported episode_reward_max=38.27290922217071,episode_reward_min=29.208372473716736,episode_reward_mean=33.81569207767025,episode_len_mean=100.0,episodes_this_iter=8,policy_reward_min={'policy_0': 13.927396217361093, 'policy_1': 14.438492741435766},policy_reward_max={'policy_0': 19.682217812165618, 'policy_1': 19.721824208274484},policy_reward_mean={'policy_0': 16.94444537024945, 'policy_1': 16.871246707420795},custom_metrics={},sampler_perf={'mean_env_wait_ms': 0.14409193051810987, 'mean_raw_obs_processing_ms': 0.24032074687145258, 'mean_inference_ms': 2.0443635653029606, 'mean_action_processing_ms': 0.07901005875759307},off_policy_estimator={},num_healthy_workers=8,timers={'learn_time_ms': 11.791, 'learn_throughput': 2713.904, 'update_time_ms': 2.415},info={'learner': {'policy_0': {'allreduce_latency': 0.0, 'grad_gnorm': array(0.00399148, dtype=float32), 'cur_lr': 0.15, 'mean_q': 4.294923782348633, 'min_q': 4.068885803222656, 'max_q': 4.38558149337768

...

(pid=94331)   z = torch.range(
(pid=94331) /Users/matiascovarrubias/Library/Caches/pypoetry/virtualenvs/marketsai-reVLCGV_-py3.8/lib/python3.8/site-packages/ray/rllib/agents/dqn/dqn_torch_policy.py:357: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
(pid=94331)   support_prob_per_action = nn.functional.softmax(
(pid=94330) /Users/matiascovarrubias/Library/Caches/pypoetry/virtualenvs/marketsai-reVLCGV_-py3.8/lib/python3.8/site-packages/ray/rllib/agents/dqn/dqn_torch_model.py:145: UserWarning: torch.range is deprecated and will be removed in a future release because its behavior is inconsistent with Python's range builtin. Instead, use torch.arange, which produces values in [start, end).
(pid=94330)   z = torch.range(
(pid=94330) /Users/matiascovarrubias/Library/Caches/pypoetry/virtualenvs/marketsai-reVLCGV_-py3.8/lib/python3.8/site-packages/ray/rllib/agents/dqn/dqn_torch_policy.py:357: UserWarning: Implicit dimens

: torch.range is deprecated and will be removed in a future release because its behavior is inconsistent with Python's range builtin. Instead, use torch.arange, which produces values in [start, end).
(pid=94331)   z = torch.range(
(pid=94331) /Users/matiascovarrubias/Library/Caches/pypoetry/virtualenvs/marketsai-reVLCGV_-py3.8/lib/python3.8/site-packages/ray/rllib/agents/dqn/dqn_torch_policy.py:357: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
(pid=94331)   support_prob_per_action = nn.functional.softmax(
(pid=94330) /Users/matiascovarrubias/Library/Caches/pypoetry/virtualenvs/marketsai-reVLCGV_-py3.8/lib/python3.8/site-packages/ray/rllib/agents/dqn/dqn_torch_model.py:145: UserWarning: torch.range is deprecated and will be removed in a future release because its behavior is inconsistent with Python's range builtin. Instead, use torch.arange, which produces values in [start, end).
(pid=94330)   z = torch.range(


(pid=94331)   z = torch.range(
(pid=94331) /Users/matiascovarrubias/Library/Caches/pypoetry/virtualenvs/marketsai-reVLCGV_-py3.8/lib/python3.8/site-packages/ray/rllib/agents/dqn/dqn_torch_policy.py:357: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
(pid=94331)   support_prob_per_action = nn.functional.softmax(
(pid=94330) /Users/matiascovarrubias/Library/Caches/pypoetry/virtualenvs/marketsai-reVLCGV_-py3.8/lib/python3.8/site-packages/ray/rllib/agents/dqn/dqn_torch_model.py:145: UserWarning: torch.range is deprecated and will be removed in a future release because its behavior is inconsistent with Python's range builtin. Instead, use torch.arange, which produces values in [start, end).
(pid=94330)   z = torch.range(
(pid=94330) /Users/matiascovarrubias/Library/Caches/pypoetry/virtualenvs/marketsai-reVLCGV_-py3.8/lib/python3.8/site-packages/ray/rllib/agents/dqn/dqn_torch_policy.py:357: UserWarning: Implicit dimens

 deprecated and will be removed in a future release because its behavior is inconsistent with Python's range builtin. Instead, use torch.arange, which produces values in [start, end).
(pid=94331)   z = torch.range(
(pid=94331) /Users/matiascovarrubias/Library/Caches/pypoetry/virtualenvs/marketsai-reVLCGV_-py3.8/lib/python3.8/site-packages/ray/rllib/agents/dqn/dqn_torch_policy.py:357: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
(pid=94331)   support_prob_per_action = nn.functional.softmax(
(pid=94330) /Users/matiascovarrubias/Library/Caches/pypoetry/virtualenvs/marketsai-reVLCGV_-py3.8/lib/python3.8/site-packages/ray/rllib/agents/dqn/dqn_torch_model.py:145: UserWarning: torch.range is deprecated and will be removed in a future release because its behavior is inconsistent with Python's range builtin. Instead, use torch.arange, which produces values in [start, end).
(pid=94330)   z = torch.range(
(pid=94330) /Use

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
DQN_diffdemand_dbe1e_00000,TERMINATED,,40,107.118,40000,25.8598,30.5765,19.605,100


In [6]:
#Policy Gradient Methods: PG, A2C, A3C, PPO, APPO

# algo_list=["PG", "A2C", "A3C", "PPO", "APPO"]
algo_list=["PG", "PPO"]
for i in range(len(algo_list)):
    exp_name = exp_label + algo_list[i]
    results = tune.run(
        algo_list[i],
        name=exp_name,
        config=training_config,
        #checkpoint_freq=250,
        checkpoint_at_end=True,
        stop=stop,
        callbacks=[MLflowLoggerCallback(experiment_name=exp_name, save_artifact=True)],
        verbose=verbosity
    )


(pid=94477) /Users/matiascovarrubias/Library/Caches/pypoetry/virtualenvs/marketsai-reVLCGV_-py3.8/lib/python3.8/site-packages/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
(pid=94477)   warnings.warn(msg)
(pid=94477) WARNING:tensorflow:From /Users/matiascovarrubias/Library/Caches/pypoetry/virtualenvs/marketsai-reVLCGV_-py3.8/lib/python3.8/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=94477) Instructions for updating:
(pid=94477) non-resource variables are not supported in the long term
(pid=94477) 2021-04-18 16:46:31,512	INFO trainer.py:641 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=94477) {'agent_0': array([1.6, 1.6], dtype=float32), 'agen

Trial PG_diffdemand_24f0e_00000 reported episode_reward_max=40.319497145712376,episode_reward_min=28.67565298639238,episode_reward_mean=32.91753894428257,episode_len_mean=100.0,episodes_this_iter=16,policy_reward_min={'policy_0': 13.883805776014924, 'policy_1': 14.224987832829356},policy_reward_max={'policy_0': 19.51743401028216, 'policy_1': 20.802063135430217},policy_reward_mean={'policy_0': 16.433713921462186, 'policy_1': 16.483825022820383},custom_metrics={},sampler_perf={'mean_env_wait_ms': 0.3314625595943288, 'mean_raw_obs_processing_ms': 0.37807093442658307, 'mean_inference_ms': 3.3897697243512845, 'mean_action_processing_ms': 0.18268783875593242},off_policy_estimator={},num_healthy_workers=8,timers={'sample_time_ms': 919.52, 'sample_throughput': 1740.038, 'learn_time_ms': 70.955, 'learn_throughput': 22549.643, 'update_time_ms': 4.54},info={'learner': {'policy_0': {'allreduce_latency': 0.0, 'policy_loss': 59.59907150268555}, 'policy_1': {'allreduce_latency': 0.0, 'policy_loss': 7

Trial PG_diffdemand_24f0e_00000 reported episode_reward_max=38.89804884046316,episode_reward_min=28.2858314961195,episode_reward_mean=33.34262293249369,episode_len_mean=100.0,episodes_this_iter=16,policy_reward_min={'policy_0': 13.95685507915914, 'policy_1': 13.898031471297145},policy_reward_max={'policy_0': 20.90498380921781, 'policy_1': 19.522957675158978},policy_reward_mean={'policy_0': 16.73736798638478, 'policy_1': 16.605254946108907},custom_metrics={},sampler_perf={'mean_env_wait_ms': 0.3471175424700456, 'mean_raw_obs_processing_ms': 0.3977705819027992, 'mean_inference_ms': 3.555829098869507, 'mean_action_processing_ms': 0.19006996518485128},off_policy_estimator={},num_healthy_workers=8,timers={'sample_time_ms': 960.576, 'sample_throughput': 1665.667, 'learn_time_ms': 74.253, 'learn_throughput': 21547.83, 'update_time_ms': 4.891},info={'learner': {'policy_0': {'allreduce_latency': 0.0, 'policy_loss': 99.96836853027344}, 'policy_1': {'allreduce_latency': 0.0, 'policy_loss': 93.707

Trial PG_diffdemand_24f0e_00000 reported episode_reward_max=38.12953993678093,episode_reward_min=27.545931236818433,episode_reward_mean=33.09848110772669,episode_len_mean=100.0,episodes_this_iter=16,policy_reward_min={'policy_0': 13.120429996401072, 'policy_1': 13.847640164196491},policy_reward_max={'policy_0': 20.090492827817798, 'policy_1': 19.522957675158978},policy_reward_mean={'policy_0': 16.597361067999156, 'policy_1': 16.50112003972754},custom_metrics={},sampler_perf={'mean_env_wait_ms': 0.35327666711853295, 'mean_raw_obs_processing_ms': 0.40836169963368407, 'mean_inference_ms': 3.6329359588513537, 'mean_action_processing_ms': 0.19198696916031474},off_policy_estimator={},num_healthy_workers=8,timers={'sample_time_ms': 965.223, 'sample_throughput': 1657.648, 'learn_time_ms': 95.114, 'learn_throughput': 16821.933, 'update_time_ms': 4.887},info={'learner': {'policy_0': {'allreduce_latency': 0.0, 'policy_loss': 122.93505859375}, 'policy_1': {'allreduce_latency': 0.0, 'policy_loss': 

Trial PG_diffdemand_24f0e_00000 reported episode_reward_max=39.22208006493747,episode_reward_min=26.84163029305637,episode_reward_mean=32.578108506016434,episode_len_mean=100.0,episodes_this_iter=16,policy_reward_min={'policy_0': 13.120429996401072, 'policy_1': 12.835816284641623},policy_reward_max={'policy_0': 19.964524414390326, 'policy_1': 20.612755473703146},policy_reward_mean={'policy_0': 16.36013128850609, 'policy_1': 16.21797721751034},custom_metrics={},sampler_perf={'mean_env_wait_ms': 0.3630543007213579, 'mean_raw_obs_processing_ms': 0.42417651963793346, 'mean_inference_ms': 3.744845025908439, 'mean_action_processing_ms': 0.19694909999551846},off_policy_estimator={},num_healthy_workers=8,timers={'sample_time_ms': 1055.064, 'sample_throughput': 1516.496, 'learn_time_ms': 112.718, 'learn_throughput': 14194.775, 'update_time_ms': 4.645},info={'learner': {'policy_0': {'allreduce_latency': 0.0, 'policy_loss': 82.075439453125}, 'policy_1': {'allreduce_latency': 0.0, 'policy_loss': 1

...

Trial PPO_diffdemand_40ea4_00000 reported episode_reward_max=37.22913474775851,episode_reward_min=28.46638896688819,episode_reward_mean=33.227281753234564,episode_len_mean=100.0,episodes_this_iter=48,policy_reward_min={'policy_0': 13.321194356307387, 'policy_1': 14.643613064661622},policy_reward_max={'policy_0': 18.289578830823302, 'policy_1': 20.25048174895346},policy_reward_mean={'policy_0': 16.254929032735525, 'policy_1': 16.97235272049904},custom_metrics={},sampler_perf={'mean_env_wait_ms': 0.19378625842489058, 'mean_raw_obs_processing_ms': 0.2550955820898368, 'mean_inference_ms': 2.5152922618938613, 'mean_action_processing_ms': 0.10136809646414506},off_policy_estimator={},num_healthy_workers=8,timers={'sample_time_ms': 1880.747, 'sample_throughput': 2552.177, 'learn_time_ms': 22453.966, 'learn_throughput': 213.771, 'update_time_ms': 2.396},info={'learner': {'policy_0': {'allreduce_latency': 0.0, 'cur_kl_coeff': 1.5187499999999998, 'cur_lr': 0.15000000000000005, 'total_loss': inf, 

Trial PPO_diffdemand_40ea4_00000 reported episode_reward_max=37.340190613642335,episode_reward_min=28.725266996771097,episode_reward_mean=33.02600660093129,episode_len_mean=100.0,episodes_this_iter=48,policy_reward_min={'policy_0': 13.31330930814147, 'policy_1': 14.700338741764426},policy_reward_max={'policy_0': 18.208316691219807, 'policy_1': 20.25048174895346},policy_reward_mean={'policy_0': 16.021807844433933, 'policy_1': 17.004198756497352},custom_metrics={},sampler_perf={'mean_env_wait_ms': 0.19000749572798667, 'mean_raw_obs_processing_ms': 0.2515071968837614, 'mean_inference_ms': 2.465405696542957, 'mean_action_processing_ms': 0.09916076463559279},off_policy_estimator={},num_healthy_workers=8,timers={'sample_time_ms': 1841.146, 'sample_throughput': 2607.072, 'learn_time_ms': 22368.898, 'learn_throughput': 214.584, 'update_time_ms': 2.393},info={'learner': {'policy_0': {'allreduce_latency': 0.0, 'cur_kl_coeff': 2.2781250000000006, 'cur_lr': 0.15000000000000005, 'total_loss': inf, 

Trial PPO_diffdemand_40ea4_00000 reported episode_reward_max=37.62776047736406,episode_reward_min=28.725266996771097,episode_reward_mean=33.24965226076543,episode_len_mean=100.0,episodes_this_iter=48,policy_reward_min={'policy_0': 13.31330930814147, 'policy_1': 14.700338741764426},policy_reward_max={'policy_0': 18.66233503073454, 'policy_1': 19.809376675635576},policy_reward_mean={'policy_0': 16.06260389411822, 'policy_1': 17.187048366647215},custom_metrics={},sampler_perf={'mean_env_wait_ms': 0.18794138257524742, 'mean_raw_obs_processing_ms': 0.25015176854694393, 'mean_inference_ms': 2.4379487303251426, 'mean_action_processing_ms': 0.09799115736516682},off_policy_estimator={},num_healthy_workers=8,timers={'sample_time_ms': 1838.038, 'sample_throughput': 2611.48, 'learn_time_ms': 22715.958, 'learn_throughput': 211.305, 'update_time_ms': 2.444},info={'learner': {'policy_0': {'allreduce_latency': 0.0, 'cur_kl_coeff': 3.4171875000000003, 'cur_lr': 0.15000000000000005, 'total_loss': inf, '

Trial PPO_diffdemand_40ea4_00000 reported episode_reward_max=37.62776047736406,episode_reward_min=29.49546417593956,episode_reward_mean=33.40173818718642,episode_len_mean=100.0,episodes_this_iter=48,policy_reward_min={'policy_0': 13.440248196944594, 'policy_1': 15.066298995167017},policy_reward_max={'policy_0': 18.66233503073454, 'policy_1': 19.947984717786312},policy_reward_mean={'policy_0': 16.083956395424902, 'policy_1': 17.31778179176152},custom_metrics={},sampler_perf={'mean_env_wait_ms': 0.18865815006832182, 'mean_raw_obs_processing_ms': 0.2524648624015312, 'mean_inference_ms': 2.446889750381646, 'mean_action_processing_ms': 0.09837995525336435},off_policy_estimator={},num_healthy_workers=8,timers={'sample_time_ms': 1856.807, 'sample_throughput': 2585.083, 'learn_time_ms': 23348.953, 'learn_throughput': 205.577, 'update_time_ms': 2.474},info={'learner': {'policy_0': {'allreduce_latency': 0.0, 'cur_kl_coeff': 5.125781249999998, 'cur_lr': 0.15000000000000005, 'total_loss': inf, 'po

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_diffdemand_40ea4_00000,TERMINATED,,9,226.928,43200,33.4017,37.6278,29.4955,100


In [7]:
#Mixed:  SAC, IMPALA

algo_list=["IMPALA"]
for i in range(len(algo_list)):
    exp_name = exp_label + algo_list[i]
    results = tune.run(
        algo_list[i],
        name=exp_name,
        config=training_config,
        #checkpoint_freq=250,
        checkpoint_at_end=True,
        stop=stop,
        callbacks=[MLflowLoggerCallback(experiment_name=exp_name, save_artifact=True)],
        verbose=verbosity
    )

(pid=95024) /Users/matiascovarrubias/Library/Caches/pypoetry/virtualenvs/marketsai-reVLCGV_-py3.8/lib/python3.8/site-packages/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
(pid=95024)   warnings.warn(msg)
(pid=95024) WARNING:tensorflow:From /Users/matiascovarrubias/Library/Caches/pypoetry/virtualenvs/marketsai-reVLCGV_-py3.8/lib/python3.8/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=95024) Instructions for updating:
(pid=95024) non-resource variables are not supported in the long term
(pid=95024) 2021-04-18 16:51:19,852	INFO trainer.py:641 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=95024) {'agent_0': array([1.6, 1.6], dtype=float32), 'agen

Trial IMPALA_diffdemand_d156b_00000 reported episode_reward_max=37.69141532666981,episode_reward_min=27.577166752889752,episode_reward_mean=33.044447469447086,episode_len_mean=100.0,episodes_this_iter=227,policy_reward_min={'policy_0': 13.08484984561801, 'policy_1': 12.987941138446331},policy_reward_max={'policy_0': 20.03181435726583, 'policy_1': 20.05359734222293},policy_reward_mean={'policy_0': 16.47280166509032, 'policy_1': 16.571645804356766},custom_metrics={},sampler_perf={'mean_env_wait_ms': 0.27469121787088663, 'mean_raw_obs_processing_ms': 0.34797687335191135, 'mean_inference_ms': 2.801560835155703, 'mean_action_processing_ms': 0.14293357931173617},off_policy_estimator={},num_healthy_workers=8,timers={'sample_time_ms': 228.773, 'sample_throughput': 2185.576},info={'num_steps_sampled': 23750, 'num_steps_trained': 23500, 'num_weight_broadcasts': 43, 'learner_queue': {'size_count': 47, 'size_mean': 0.10638297872340426, 'size_std': 0.30832716481254135, 'size_quantiles': [0.0, 0.0, 

Trial IMPALA_diffdemand_d156b_00000 reported episode_reward_max=37.100733142346144,episode_reward_min=26.628898495808244,episode_reward_mean=32.04248508223955,episode_len_mean=100.0,episodes_this_iter=211,policy_reward_min={'policy_0': 12.791369140148163, 'policy_1': 12.988685317337513},policy_reward_max={'policy_0': 18.86882996559143, 'policy_1': 19.26854708790779},policy_reward_mean={'policy_0': 16.07923711236054, 'policy_1': 15.963247969879005},custom_metrics={},sampler_perf={'mean_env_wait_ms': 0.28405615888665237, 'mean_raw_obs_processing_ms': 0.36925705254859525, 'mean_inference_ms': 2.893535938145321, 'mean_action_processing_ms': 0.14677819907369782},off_policy_estimator={},num_healthy_workers=8,timers={'sample_time_ms': 258.98, 'sample_throughput': 1930.648},info={'num_steps_sampled': 44850, 'num_steps_trained': 44500, 'num_weight_broadcasts': 84, 'learner_queue': {'size_count': 89, 'size_mean': 0.02, 'size_std': 0.13999999999999999, 'size_quantiles': [0.0, 0.0, 0.0, 0.0, 1.0]}

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
IMPALA_diffdemand_d156b_00000,TERMINATED,,3,21.4717,44850,32.0425,37.1007,26.6289,100


### Continuous Space


In [8]:
#DDGP uses its own exploration config
# See exploration config in https://github.com/ray-project/ray/blob/master/rllib/utils/exploration/ornstein_uhlenbeck_noise.pyDDPG 
exploration_config_cont = {
        # DDPG uses OrnsteinUhlenbeck (stateful) noise to be added to NN-output
        # actions (after a possible pure random phase of n timesteps).
        "type": "OrnsteinUhlenbeckNoise",
        "final_scale": 0.02,
        "scale_timesteps": 100000,
    }

training_config_cont=training_config.copy()
env_config_cont=env_config.copy()
training_config_cont["exploration_config"] = exploration_config_cont
env_config_cont["mkt_config"]["space_type"] = "Continuous"

env=DiffDemand(env_config_cont)
training_config_cont["env_config"] = env_config_cont
training_config_cont["multiagent"]["policies"] =  {
            policy_ids[i]: (None, env.observation_space[f"agent_{i}"], env.action_space[f"agent_{i}"],{},) for i in range(env.n_agents)
}
#print(env_config)
print(training_config_cont)
print(env.action_space)


{'gamma': 0.95, 'lr': 0.15, 'env': 'diffdemand', 'exploration_config': {'type': 'OrnsteinUhlenbeckNoise', 'final_scale': 0.02, 'scale_timesteps': 100000}, 'env_config': {'mkt_config': {'parameteres': {'cost': [1, 1], 'values': [2, 2], 'ext_demand': 0, 'substitution': 0.25}, 'space_type': 'Continuous', 'gridpoints': 16}}, 'horizon': 100, 'soft_horizon': True, 'no_done_at_end': True, 'multiagent': {'policies': {'policy_0': (None, Box(1.0, 2.0, (2,), float32), Box(1.0, 2.0, (1,), float32), {}), 'policy_1': (None, Box(1.0, 2.0, (2,), float32), Box(1.0, 2.0, (1,), float32), {})}, 'policy_mapping_fn': <function <lambda> at 0x19d187550>}, 'framework': 'torch', 'num_workers': 8, 'num_gpus': 0, 'timesteps_per_iteration': 1000, 'normalize_actions': False}
{'agent_0': Box(1.0, 2.0, (1,), float32), 'agent_1': Box(1.0, 2.0, (1,), float32)}


In [9]:
#COntinuous action space DQN

#algo_list=["DDPG", "TD3", "SAC"]
algo_list=["TD3"]
for i in range(len(algo_list)):
    exp_name = exp_label + "_cont_" + algo_list[i]
    results = tune.run(
        algo_list[i],
        name=exp_name,
        config=training_config_cont,
        #checkpoint_freq=250,
        checkpoint_at_end=True,
        stop=stop,
        callbacks=[MLflowLoggerCallback(experiment_name=exp_name, save_artifact=True)],
        verbose=verbosity
    )

tualenvs/marketsai-reVLCGV_-py3.8/lib/python3.8/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=95108) Instructions for updating:
(pid=95108) non-resource variables are not supported in the long term
(pid=95111) WARNING:tensorflow:From /Users/matiascovarrubias/Library/Caches/pypoetry/virtualenvs/marketsai-reVLCGV_-py3.8/lib/python3.8/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=95111) Instructions for updating:
(pid=95111) non-resource variables are not supported in the long term
(pid=95113) WARNING:tensorflow:From /Users/matiascovarrubias/Library/Caches/pypoetry/virtualenvs/marketsai-reVLCGV_-py3.8/lib/python3.8/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflo

Trial TD3_diffdemand_e4b6e_00000 reported episode_reward_max=[55.317818],episode_reward_min=[30.909946],episode_reward_mean=37.40788269042969,episode_len_mean=100.0,episodes_this_iter=8,policy_reward_min={'policy_0': 14.85215, 'policy_1': 15.047528},policy_reward_max={'policy_0': 28.826424, 'policy_1': 27.028486},policy_reward_mean={'policy_0': 18.612354, 'policy_1': 18.79553},custom_metrics={},sampler_perf={'mean_env_wait_ms': 0.5341949720639467, 'mean_raw_obs_processing_ms': 1.5231993581504173, 'mean_inference_ms': 5.603019277395738, 'mean_action_processing_ms': 0.4778903112123763},off_policy_estimator={},num_healthy_workers=8,timers={'learn_time_ms': 38.857, 'learn_throughput': 2573.553, 'update_time_ms': 5.227},info={'learner': {'policy_0': {'allreduce_latency': 0.0, 'actor_loss': -0.48416560888290405, 'critic_loss': 0.013488397002220154, 'mean_q': 0.43282875418663025, 'max_q': 0.5108005404472351, 'min_q': 0.3258611261844635, 'mean_td_error': -0.022518331184983253, 'td_error': arra

Trial TD3_diffdemand_e4b6e_00000 reported episode_reward_max=[67.01797],episode_reward_min=[30.909946],episode_reward_mean=42.45185470581055,episode_len_mean=100.0,episodes_this_iter=16,policy_reward_min={'policy_0': 14.85215, 'policy_1': 15.047528},policy_reward_max={'policy_0': 34.32957, 'policy_1': 34.311802},policy_reward_mean={'policy_0': 21.12885, 'policy_1': 21.32301},custom_metrics={},sampler_perf={'mean_env_wait_ms': 0.5269504665004628, 'mean_raw_obs_processing_ms': 1.510794443236814, 'mean_inference_ms': 5.553729312525707, 'mean_action_processing_ms': 0.47276108841532327},off_policy_estimator={},num_healthy_workers=8,timers={'learn_time_ms': 32.081, 'learn_throughput': 3117.135, 'update_time_ms': 4.057},info={'learner': {'policy_0': {'allreduce_latency': 0.0, 'actor_loss': -0.6249244809150696, 'critic_loss': 0.012099440209567547, 'mean_q': 0.5923510193824768, 'max_q': 0.6374791860580444, 'min_q': 0.5372205972671509, 'mean_td_error': 0.015311526134610176, 'td_error': array([[-

Trial TD3_diffdemand_e4b6e_00000 reported episode_reward_max=[67.01797],episode_reward_min=[30.909946],episode_reward_mean=44.87808609008789,episode_len_mean=100.0,episodes_this_iter=8,policy_reward_min={'policy_0': 14.85215, 'policy_1': 15.047528},policy_reward_max={'policy_0': 34.32957, 'policy_1': 34.311802},policy_reward_mean={'policy_0': 22.35478, 'policy_1': 22.523308},custom_metrics={},sampler_perf={'mean_env_wait_ms': 0.5216021867173326, 'mean_raw_obs_processing_ms': 1.4998815627477007, 'mean_inference_ms': 5.518683322425244, 'mean_action_processing_ms': 0.4686471190555464},off_policy_estimator={},num_healthy_workers=8,timers={'learn_time_ms': 28.324, 'learn_throughput': 3530.541, 'update_time_ms': 3.669},info={'learner': {'policy_0': {'allreduce_latency': 0.0, 'actor_loss': -0.7001399397850037, 'critic_loss': 0.01563209295272827, 'mean_q': 0.684307873249054, 'max_q': 0.705432116985321, 'min_q': 0.6508501172065735, 'mean_td_error': 0.042333997786045074, 'td_error': array([[ 0.1

Trial TD3_diffdemand_e4b6e_00000 reported episode_reward_max=[67.01797],episode_reward_min=[30.909946],episode_reward_mean=47.26375961303711,episode_len_mean=100.0,episodes_this_iter=8,policy_reward_min={'policy_0': 14.85215, 'policy_1': 15.047528},policy_reward_max={'policy_0': 34.32957, 'policy_1': 34.311802},policy_reward_mean={'policy_0': 23.548517, 'policy_1': 23.715256},custom_metrics={},sampler_perf={'mean_env_wait_ms': 0.5144776531644758, 'mean_raw_obs_processing_ms': 1.484210716179362, 'mean_inference_ms': 5.4685463710100155, 'mean_action_processing_ms': 0.46301176668970273},off_policy_estimator={},num_healthy_workers=8,timers={'learn_time_ms': 28.415, 'learn_throughput': 3519.317, 'update_time_ms': 4.112},info={'learner': {'policy_0': {'allreduce_latency': 0.0, 'actor_loss': -0.7190511226654053, 'critic_loss': 0.012456849217414856, 'mean_q': 0.7134852409362793, 'max_q': 0.7376866340637207, 'min_q': 0.6997673511505127, 'mean_td_error': -0.024501826614141464, 'td_error': array(

...

Trial TD3_diffdemand_e4b6e_00000 reported episode_reward_max=[66.9377],episode_reward_min=[66.7094],episode_reward_mean=66.8021240234375,episode_len_mean=100.0,episodes_this_iter=8,policy_reward_min={'policy_0': 32.613594, 'policy_1': 32.84972},policy_reward_max={'policy_0': 33.992184, 'policy_1': 34.25237},policy_reward_mean={'policy_0': 33.36666, 'policy_1': 33.43549},custom_metrics={},sampler_perf={'mean_env_wait_ms': 0.3259736590507081, 'mean_raw_obs_processing_ms': 0.9334907572821917, 'mean_inference_ms': 3.4764496321754463, 'mean_action_processing_ms': 0.29110345463956105},off_policy_estimator={},num_healthy_workers=8,timers={'learn_time_ms': 26.941, 'learn_throughput': 3711.79, 'update_time_ms': 2.784},info={'learner': {'policy_0': {'allreduce_latency': 0.0, 'actor_loss': -3.1260459423065186, 'critic_loss': 0.009155292063951492, 'mean_q': 3.1226043701171875, 'max_q': 3.1285934448242188, 'min_q': 3.0983266830444336, 'mean_td_error': -0.042557958513498306, 'td_error': array([[-0.0

Trial TD3_diffdemand_e4b6e_00000 reported episode_reward_max=[66.9377],episode_reward_min=[66.7094],episode_reward_mean=66.7999038696289,episode_len_mean=100.0,episodes_this_iter=8,policy_reward_min={'policy_0': 32.613594, 'policy_1': 32.84972},policy_reward_max={'policy_0': 33.992184, 'policy_1': 34.25237},policy_reward_mean={'policy_0': 33.372112, 'policy_1': 33.42783},custom_metrics={},sampler_perf={'mean_env_wait_ms': 0.3224145240716643, 'mean_raw_obs_processing_ms': 0.923537518151328, 'mean_inference_ms': 3.441528174499893, 'mean_action_processing_ms': 0.2880014698801674},off_policy_estimator={},num_healthy_workers=8,timers={'learn_time_ms': 26.616, 'learn_throughput': 3757.189, 'update_time_ms': 2.764},info={'learner': {'policy_0': {'allreduce_latency': 0.0, 'actor_loss': -3.2439920902252197, 'critic_loss': 0.005563053768128157, 'mean_q': 3.2431552410125732, 'max_q': 3.2744600772857666, 'min_q': 3.2290656566619873, 'mean_td_error': -0.011300750076770782, 'td_error': array([[ 0.22

Trial TD3_diffdemand_e4b6e_00000 reported episode_reward_max=[66.9377],episode_reward_min=[66.69294],episode_reward_mean=66.79962921142578,episode_len_mean=100.0,episodes_this_iter=8,policy_reward_min={'policy_0': 32.71598, 'policy_1': 32.84972},policy_reward_max={'policy_0': 33.992184, 'policy_1': 34.183483},policy_reward_mean={'policy_0': 33.373768, 'policy_1': 33.425884},custom_metrics={},sampler_perf={'mean_env_wait_ms': 0.3188451856528359, 'mean_raw_obs_processing_ms': 0.9137202341373698, 'mean_inference_ms': 3.4033153950784527, 'mean_action_processing_ms': 0.2847134173851573},off_policy_estimator={},num_healthy_workers=8,timers={'learn_time_ms': 27.4, 'learn_throughput': 3649.661, 'update_time_ms': 2.759},info={'learner': {'policy_0': {'allreduce_latency': 0.0, 'actor_loss': -3.3249940872192383, 'critic_loss': 0.006591449957340956, 'mean_q': 3.3249869346618652, 'max_q': 3.3385655879974365, 'min_q': 3.3115358352661133, 'mean_td_error': -0.009465793147683144, 'td_error': array([[-0

Trial TD3_diffdemand_e4b6e_00000 reported episode_reward_max=[66.9377],episode_reward_min=[66.69294],episode_reward_mean=66.79241943359375,episode_len_mean=100.0,episodes_this_iter=16,policy_reward_min={'policy_0': 32.71598, 'policy_1': 32.878235},policy_reward_max={'policy_0': 33.906063, 'policy_1': 34.183483},policy_reward_mean={'policy_0': 33.37019, 'policy_1': 33.422256},custom_metrics={},sampler_perf={'mean_env_wait_ms': 0.31234081388932533, 'mean_raw_obs_processing_ms': 0.8957250975944093, 'mean_inference_ms': 3.334848620606812, 'mean_action_processing_ms': 0.27887910368658286},off_policy_estimator={},num_healthy_workers=8,timers={'learn_time_ms': 27.551, 'learn_throughput': 3629.584, 'update_time_ms': 2.876},info={'learner': {'policy_0': {'allreduce_latency': 0.0, 'actor_loss': -3.3962795734405518, 'critic_loss': 0.006818897090852261, 'mean_q': 3.394848346710205, 'max_q': 3.408041000366211, 'min_q': 3.3779964447021484, 'mean_td_error': -0.018629733473062515, 'td_error': array([[

Trial name,status,loc,iter,total time (s),ts,reward,episode_len_mean,episodes_this_iter,num_healthy_workers
TD3_diffdemand_e4b6e_00000,TERMINATED,,31,249.579,40000,66.7924,100,16,8


In [10]:
shutdown()